In [2]:
!pip install trl -q

In [3]:
import os
import random
import gc
import torch
import torch.nn as nn
import numpy as np
import wandb
import pandas as pd
import datasets
import torch.nn.functional as F

from collections import deque
from dataclasses import dataclass
from datasets import load_dataset
from typing import Any, Optional, Union
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification,PreTrainedTokenizerBase, OPTForSequenceClassification
from trl import SFTTrainer, SFTConfig, RewardTrainer, RewardConfig
from transformers.utils import PaddingStrategy


In [4]:
@dataclass
class CFG:
    seed: int = 42

In [5]:
def seed_env(seed: int = CFG.seed) -> None:
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

def seed_torch(seed: int = CFG.seed) -> None:
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

def seed_everything() -> None:
  """Set seeds"""
  seed_torch()
  seed_env()

def init_wandb() -> None:
  wandb.init(project='Reinforce_RLHF', entity='lulim')

seed_everything()
init_wandb()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
train_dataset = load_dataset("nvidia/HelpSteer2", split="train")
validation_dataset = load_dataset("nvidia/HelpSteer2", split="validation")

README.md:   0%|          | 0.00/25.0k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/11.3M [00:00<?, ?B/s]

validation.jsonl.gz:   0%|          | 0.00/582k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1038 [00:00<?, ? examples/s]

In [7]:
chosen_train_dataset = train_dataset.select(range(0, len(train_dataset), 2))
chosen_validation_dataset = validation_dataset.select(range(0, len(validation_dataset), 2))

In [8]:
chosen_train_dataset = chosen_train_dataset.select_columns(['response'])
chosen_train_dataset = chosen_train_dataset.rename_column('response', 'text')


chosen_validation_dataset = chosen_validation_dataset.select_columns(['response'])
chosen_validation_dataset = chosen_validation_dataset.rename_column('response', 'text')

chosen_train_dataset

Dataset({
    features: ['text'],
    num_rows: 10162
})

In [10]:
model_name = "facebook/opt-350m"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [11]:
training_arg = SFTConfig(
    max_seq_length=768, #try 768
    dataset_batch_size=32,
    output_dir='/kaggle/working/',
    learning_rate=1e-4,
    num_train_epochs=1,
    fp16=True
)

trainer = SFTTrainer(
    model=model,
    args=training_arg,
    train_dataset=chosen_train_dataset,
    eval_dataset=chosen_validation_dataset,
    tokenizer=tokenizer,
)
trainer.train()

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,2.084400
1000,1.952900


TrainOutput(global_step=1271, training_loss=1.9844890267336865, metrics={'train_runtime': 2387.7517, 'train_samples_per_second': 4.256, 'train_steps_per_second': 0.532, 'total_flos': 1.1213136469229568e+16, 'train_loss': 1.9844890267336865, 'epoch': 1.0})

In [ ]:
tokenized = tokenizer(text=chosen_validation_dataset['text'][0], truncation=True, padding='max_length',
                                                  max_length=768, return_tensors='pt').to(device)
input_ids, attention_mask = tokenized['input_ids'], tokenized['attention_mask']
response = policy.generate(input_ids=input_ids,
                           attention_mask=attention_mask, 
                           max_new_tokens=20,
                           top_p=0.95,
                           do_sample=True,
                           num_return_sequences=1).cpu()
print(chosen_validation_dataset['text'][0])
print('============')
print(response)

In [12]:
trainer.save_model('/kaggle/working/model_finetuned')

In [17]:
test_model = AutoModelForCausalLM.from_pretrained('/kaggle/working/model_finetuned').to('cuda')
tokenized = tokenizer(text=chosen_validation_dataset['text'][0], truncation=True, padding='max_length',
                                                  max_length=768, return_tensors='pt')
input_ids, attention_mask = tokenized['input_ids'].cuda(), tokenized['attention_mask'].cuda()
response = test_model.generate(input_ids=input_ids,
                           attention_mask=attention_mask, 
                           max_new_tokens=20,
                           top_p=0.95,
                           do_sample=True,
                           num_return_sequences=1).cpu()
print(chosen_validation_dataset['text'][0])
print('============')
print(tokenizer.batch_decode(response, skip_special_tokens=True))

In the context of NoSQL databases, master-slave replication refers to a configuration where a single master node writes data, and one or more slave nodes read data from the master and replicate it to provide read scalability. The master node is responsible for accepting write requests and updating its own data, while the slave nodes are responsible for replicating the data from the master and serving read requests.

In this configuration, the master node is the only node that can make changes to the data, while the slave nodes can only read the data and replicate it. This ensures that the data is consistent across all nodes and that there is no data loss in case of a failure of a single node.

Master-slave replication in NoSQL databases can be implemented using various technologies, such as Apache Cassandra, MongoDB, and Apache HBase. In each of these databases, the master-slave replication configuration can be set up using specific configuration options and parameters.

It's worth not

In [18]:
!zip -r /kaggle/working/model_finetuned_3.zip /kaggle/working/model_finetuned

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/model_finetuned/ (stored 0%)
  adding: kaggle/working/model_finetuned/merges.txt (deflated 53%)
  adding: kaggle/working/model_finetuned/vocab.json (deflated 59%)
  adding: kaggle/working/model_finetuned/model.safetensors (deflated 7%)
  adding: kaggle/working/model_finetuned/tokenizer_config.json (deflated 62%)
  adding: kaggle/working/model_finetuned/generation_config.json (deflated 29%)
  adding: kaggle/working/model_finetuned/config.json (deflated 48%)
  adding: kaggle/working/model_finetuned/tokenizer.json (deflated 82%)
  adding: kaggle/working/model_finetuned/special_tokens_map.json (deflated 79%)
  adding: kaggle/working/model_finetuned/training_args.bin (deflated 52%)


In [19]:
import os 
from IPython.display import FileLink


os.chdir(r'/kaggle/working')
FileLink(r'model_finetuned_3.zip')

/kaggle/working/model_finetuned_3.zip

## Reward model part

In [8]:
model_sft = AutoModelForCausalLM.from_pretrained('/kaggle/input/sft-model/kaggle/working/model_finetune')
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/sft-model/kaggle/working/model_finetuned')

In [7]:
reward_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned', 
                                                               num_labels=1)
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned')

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/sft-model-3/kaggle/working/model_finetuned and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def preprocess_function(examples):
    new_examples = {
        "input_ids_chosen": [],
        "attention_mask_chosen": [],
        "input_ids_rejected": [],
        "attention_mask_rejected": [],
    }
    for question, response_j, response_k in zip(examples["prompt"], examples["chosen"], examples["rejected"]):
        tokenized_j = tokenizer("Question: " + question + "\n\nAnswer: " + response_j, truncation=True, padding='max_length', max_length=512)
        tokenized_k = tokenizer("Question: " + question + "\n\nAnswer: " + response_k, truncation=True, padding='max_length', max_length=512)
        
        new_examples["input_ids_chosen"].append(tokenized_j["input_ids"])
        new_examples["attention_mask_chosen"].append(tokenized_j["attention_mask"])
        new_examples["input_ids_rejected"].append(tokenized_k["input_ids"])
        new_examples["attention_mask_rejected"].append(tokenized_k["attention_mask"])

    return new_examples

In [9]:
train_reward_dataset = {
    'prompt': [],
    'chosen': [],
    'rejected': []
}

validation_reward_dataset = {
    'prompt': [],
    'chosen': [],
    'rejected': []
}

for i, row in enumerate(train_dataset):
    if i % 2 == 0:
        train_reward_dataset['prompt'].append(row['prompt'])
        train_reward_dataset['chosen'].append(row['response'])
    else:
        train_reward_dataset['rejected'].append(row['response'])

train_reward_dataset = pd.DataFrame(train_reward_dataset)
train_reward_dataset = datasets.Dataset.from_pandas(train_reward_dataset)
train_reward_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 10162
})

In [10]:
for i, row in enumerate(validation_dataset):
    if i % 2 == 0:
        validation_reward_dataset['prompt'].append(row['prompt'])
        validation_reward_dataset['chosen'].append(row['response'])
    else:
        validation_reward_dataset['rejected'].append(row['response'])

validation_reward_dataset = pd.DataFrame(validation_reward_dataset)
validation_reward_dataset = datasets.Dataset.from_pandas(validation_reward_dataset)
validation_reward_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 519
})

In [11]:
reward_training_args = RewardConfig(
    output_dir='/kaggle/working/',
    max_length=512,
    learning_rate=1e-5,
    num_train_epochs=1,
    per_device_train_batch_size=8, # 32
    per_device_eval_batch_size=8,
    fp16=True
)

reward_trainer = RewardTrainer(
    model=reward_model,
    tokenizer=tokenizer,
    args=reward_training_args,
    train_dataset=train_reward_dataset,
    eval_dataset=validation_reward_dataset,
)
reward_trainer.train()

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/10162 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10162 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

Map:   0%|          | 0/519 [00:00<?, ? examples/s]

...

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss
500,0.711700


TrainOutput(global_step=951, training_loss=0.6867036869598613, metrics={'train_runtime': 2185.5166, 'train_samples_per_second': 3.48, 'train_steps_per_second': 0.435, 'total_flos': 0.0, 'train_loss': 0.6867036869598613, 'epoch': 1.0})

In [12]:
reward_trainer.save_model('/kaggle/working/reward_model_3')

In [13]:
!zip -r /kaggle/working/reward_model_3.zip /kaggle/working/reward_model_3

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/reward_model_3/ (stored 0%)
  adding: kaggle/working/reward_model_3/config.json (deflated 48%)
  adding: kaggle/working/reward_model_3/merges.txt (deflated 53%)
  adding: kaggle/working/reward_model_3/special_tokens_map.json (deflated 79%)
  adding: kaggle/working/reward_model_3/tokenizer_config.json (deflated 62%)
  adding: kaggle/working/reward_model_3/model.safetensors (deflated 7%)
  adding: kaggle/working/reward_model_3/tokenizer.json (deflated 82%)
  adding: kaggle/working/reward_model_3/training_args.bin (deflated 51%)
  adding: kaggle/working/reward_model_3/vocab.json (deflated 59%)


In [14]:
import os 
from IPython.display import FileLink


os.chdir(r'/kaggle/working')
FileLink(r'reward_model_3.zip')

/kaggle/working/reward_model_3.zip

## RL part

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# sft_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/sft-model/kaggle/working/model_finetuned').to(device)
sft_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned').to(device)
# reward_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/reward-model-tuned-2/kaggle/working/reward_model').to(device)
reward_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/reward-model-3/kaggle/working/reward_model_3').to(device)
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/reward-model-tuned-2/kaggle/working/reward_model')
reward_model

OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 512, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
      (project_out): Linear(in_features=1024, out_features=512, bias=False)
      (project_in): Linear(in_features=512, out_features=1024, bias=False)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Line

In [10]:
def reinforce_with_baseline(policy, tokenizer, reward_model, train_dataloader, 
                            validation_dataloader, num_epochs, gamma=0.99):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    policy.train()
    reward_model.eval()
    
    
    optimizer = torch.optim.Adam(policy.parameters(), lr=1e-6)

    
    all_rewards = []
    rewards_aggregated = []
    mx_reward = 0.0
    step = 0
    for episode in tqdm(range(num_epochs)):
        log_probs = []
        rewards = []

        for j, item in tqdm(enumerate(train_dataloader)):
            tokenized = tokenizer(text=item['text'], truncation=True, padding='max_length',
                                                  max_length=512, return_tensors='pt').to(device)
            input_ids, attention_mask = tokenized['input_ids'], tokenized['attention_mask']
            response = policy.generate(input_ids=input_ids,
                                       attention_mask=attention_mask, max_new_tokens=20,
                                       top_p=0.95,
                                       do_sample=True,
                                       num_return_sequences=1).cpu() # TODO change to top_p=0.95
            
            torch.cuda.empty_cache()
            gc.collect()
            texts_for_reward = []
            # for response_item in response:
            #     decode = [tokenizer.decode(token_ids=token_id, skip_special_tokens=True) for token_id in response_item]
            #     decoded_text = ' '.join(decode)
            #     texts_for_reward.append(decoded_text)
            texts_for_reward = tokenizer.batch_decode(response, skip_special_tokens=True)
                
            ##### get reward:
            texts_for_reward = ["Question: " + question_text + "\n\nAnswer: " + answer_text for question_text, answer_text in zip(item['text'], texts_for_reward)]
            response_tokenized = tokenizer(texts_for_reward, truncation=True, padding='max_length', max_length=512, return_tensors='pt').to(device)
            response_input_ids, response_attention_mask = response_tokenized['input_ids'], response_tokenized['attention_mask']
            with torch.no_grad():
                reward = reward_model(input_ids=response_input_ids,
                                     attention_mask=response_attention_mask)
            torch.cuda.empty_cache()
            gc.collect()
            rewards.append(reward.logits)
            all_rewards.append(reward.logits)
            ################

            ##### get log_prob:
            outputs = policy(input_ids=response_input_ids, attention_mask=response_attention_mask)
            last_token_logits = outputs.logits[:, -1, :]
            log_prob = F.log_softmax(last_token_logits, dim=-1) 
            log_probs.append(log_prob)
            ##################
            torch.cuda.empty_cache()
            gc.collect()
            ### For debug
            if j == 5:
               break

            
        n = len(rewards)
        G = 0
        discounted_rewards = deque()
        for t in reversed(range(n)):
            G = rewards[t] + gamma * G
            discounted_rewards.appendleft(G)

        del rewards
        # discounted_rewards = torch.tensor(discounted_rewards)
        discounted_rewards = torch.stack(list(discounted_rewards))
        discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)
    
    
        policy_loss = []
        baseline = torch.mean(torch.stack(all_rewards), dim=0).float()
        for discount_reward, log_prob in zip(discounted_rewards, log_probs):
            policy_loss.append(-(discount_reward - baseline) * log_prob)
        policy_loss = torch.cat(policy_loss).sum()
        policy_loss.backward()
        if j % 2 == 0:
            optimizer.step()
            optimizer.zero_grad()
        print(f'Episode: {episode}, loss: {policy_loss}, sum of rewards in batch: {torch.sum(discounted_rewards.detach().cpu().mean(dim=0))}')
        rewards_aggregated.append(torch.sum(discounted_rewards.detach().cpu().mean(dim=0)).item())
        wandb.log({
            'episode': episode,
            'loss': policy_loss,
            'sum_mean_discount_reward_batch': torch.sum(discounted_rewards.detach().cpu().mean(dim=0)).item()
        })
        if torch.sum(discounted_rewards.detach().cpu().mean(dim=0)).item() > mx_reward:
            mx_reward = torch.sum(discounted_rewards.detach().cpu().mean(dim=0)).item()
            # torch.save(policy.state_dict(), os.path.join('/kaggle/working/', 'best_policy.pth'))
            # policy.save_pretrained(os.path.join('/kaggle/working/', 'last_policy'))
        torch.cuda.empty_cache()
        gc.collect()

    policy.save_pretrained(os.path.join('/kaggle/working/', 'last_policy'))
    return rewards_aggregated, all_rewards, policy

In [11]:
# create batch and using batch
train_rl_dataset = chosen_train_dataset
train_rl_dataloader = DataLoader(train_rl_dataset, batch_size=2, shuffle=True, num_workers=4) # for test

validation_rl_dataset = chosen_validation_dataset
validation_rl_dataloader = DataLoader(validation_rl_dataset, batch_size=2, shuffle=False, num_workers=4)


In [ ]:
# TODO: train reward model on 512 max_length

rewards_aggregated, all_rewards, policy_model = reinforce_with_baseline(sft_model, tokenizer, reward_model, train_rl_dataloader,
                       validation_rl_dataloader, num_epochs=100) # for run num_epochs=100

  0%|          | 0/100 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


0it [00:00, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Episode: 0, loss: 11265570.0, sum of rewards in batch: 0.0


...

Episode: 63, loss: 5758187.0, sum of rewards in batch: 1.7881393432617188e-07


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

0it [00:00, ?it/s]

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

Episode: 64, loss: 6559419.0, sum of rewards in batch: 5.960464477539063e-08


In [21]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
!zip -r /kaggle/working/policy_result.zip /kaggle/working/last_policy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/last_policy/ (stored 0%)
  adding: kaggle/working/last_policy/config.json (deflated 48%)
  adding: kaggle/working/last_policy/model.safetensors (deflated 7%)
  adding: kaggle/working/last_policy/generation_config.json (deflated 29%)


In [14]:
import os 
from IPython.display import FileLink


os.chdir(r'/kaggle/working')
FileLink(r'policy_result.zip')

/kaggle/working/policy_result.zip

## Validate reward

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

sft_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned').to(device)
# policy_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/policy-model/kaggle/working/best_policy').to(device)
# policy_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/last-episode-policy/kaggle/working/last_policy').to(device)
reward_model = AutoModelForSequenceClassification.from_pretrained('/kaggle/input/reward-model-3/kaggle/working/reward_model_3').to(device)
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/reward-model-tuned-2/kaggle/working/reward_model')


In [9]:
policy_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/policy-result/kaggle/working/last_policy').to(device)

In [22]:
def get_reward_on_validation(model, reward_model, tokenizer, validation_dataloader, gamma=0.99):
    model.eval()
    rewards = []
    for j, item in enumerate(validation_rl_dataloader):
        tokenized = tokenizer(text=item['text'], truncation=True, padding='max_length',
                              max_length=512, return_tensors='pt').to(device)
        input_ids, attention_mask = tokenized['input_ids'], tokenized['attention_mask']
        response = model.generate(input_ids=input_ids,
                                  attention_mask=attention_mask, max_new_tokens=20,
                                  top_p=0.95,
                                  do_sample=True,
                                  num_return_sequences=1).cpu()
        
        torch.cuda.empty_cache()
        gc.collect()
        
        texts_for_reward = []
        for response_item in response:
            decode = [tokenizer.decode(token_ids=token_id, skip_special_tokens=True) for token_id in response_item]
            decoded_text = ' '.join(decode)
            if j == 0:
                print(decoded_text)
            texts_for_reward.append(decoded_text)
            
        ##### get reward:
        texts_for_reward = [answer_text for question_text, answer_text in zip(item['text'], texts_for_reward)]
        response_tokenized = tokenizer(texts_for_reward, truncation=True, padding='max_length', max_length=512, return_tensors='pt').to(device)
        response_input_ids, response_attention_mask = response_tokenized['input_ids'], response_tokenized['attention_mask']
        with torch.no_grad():
            reward = reward_model(input_ids=response_input_ids,
                                 attention_mask=response_attention_mask)
        torch.cuda.empty_cache()
        gc.collect()

        rewards.append(reward.logits)
        
        if j == 6:
            break

    n = len(rewards)
    G = 0
    discounted_rewards = deque()
    for t in reversed(range(n)):
        G = rewards[t] + gamma * G
        discounted_rewards.appendleft(G)

    # discounted_rewards = torch.tensor(discounted_rewards)
    discounted_rewards = torch.stack(list(discounted_rewards))
    discounted_rewards = (discounted_rewards - discounted_rewards.mean()) / (discounted_rewards.std() + 1e-9)

    return discounted_rewards, texts_for_reward

In [23]:
sft_reward, texts_for_reward_sft = get_reward_on_validation(sft_model, reward_model, tokenizer,
                                      validation_rl_dataloader)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 In  the  context  of  No SQL  databases ,  master - slave  replication  refers  to  a  configuration  where  a  single  master  node  writes  data ,  and  one  or  more  slave  nodes  read  data  from  the  master  and  replicate  it  to  provide  read  scal ability .  The  master  node  is  responsible  for  accepting  write  requests  and  updating  its  own  data ,  while  the  slave  nodes  are  responsible  for  repl icating  the  data  from  the  master  and  serving  read  requests . 
 
 In  this  configuration ,  the  master  node  is  the  only  node  that  can  make  changes  to  the  data ,  while  the  slave  nodes  can  only  read  the  data  and  replicate  it .  This  ensures  that  the  data  is  consistent  across  all  nodes  and  that  there  is  no  data  loss  in  case  of  a  failure  of  a  single  node . 
 
 Master - slave  replication  in  No SQL  databases  can  be  implemented  using  various  technologies ,  such  as  Apache  Cassandra ,  Mongo DB ,  and  A

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [24]:
policy_reward, texts_for_reward_policy = get_reward_on_validation(policy_model, reward_model, tokenizer,
                                         validation_rl_dataloader)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

 In  the  context  of  No SQL  databases ,  master - slave  replication  refers  to  a  configuration  where  a  single  master  node  writes  data ,  and  one  or  more  slave  nodes  read  data  from  the  master  and  replicate  it  to  provide  read  scal ability .  The  master  node  is  responsible  for  accepting  write  requests  and  updating  its  own  data ,  while  the  slave  nodes  are  responsible  for  repl icating  the  data  from  the  master  and  serving  read  requests . 
 
 In  this  configuration ,  the  master  node  is  the  only  node  that  can  make  changes  to  the  data ,  while  the  slave  nodes  can  only  read  the  data  and  replicate  it .  This  ensures  that  the  data  is  consistent  across  all  nodes  and  that  there  is  no  data  loss  in  case  of  a  failure  of  a  single  node . 
 
 Master - slave  replication  in  No SQL  databases  can  be  implemented  using  various  technologies ,  such  as  Apache  Cassandra ,  Mongo DB ,  and  A

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='le

In [25]:
texts_for_reward_sft

[' The  moment  of  inertia  of  a  tra pez ium  can  be  calculated  using  the  following  equation : \n \n I  =  ( 1 / 12 )  *  ( base  *  height ) 2  *  ( per imeter  -  ( base  +  top )) \n \n where : \n \n I  =  moment  of  inertia \n base  =  the  length  of  the  base  of  the  tra pez ium \n height  =  the  height  of  the  tra pez ium \n per imeter  =  the  perimeter  of  the  tra pez ium \n top  =  the  length  of  the  top  of  the  tra pez ium                                                                                                                                                                                                                                                                                                                                                                                                                           \n base  =  the  height  of  the  bottom  of  the  tra pez ium \n \n where : \n \n I',
 ' Sure ,  here  are  two  questions  with

In [26]:
texts_for_reward_policy

[' The  moment  of  inertia  of  a  tra pez ium  can  be  calculated  using  the  following  equation : \n \n I  =  ( 1 / 12 )  *  ( base  *  height ) 2  *  ( per imeter  -  ( base  +  top )) \n \n where : \n \n I  =  moment  of  inertia \n base  =  the  length  of  the  base  of  the  tra pez ium \n height  =  the  height  of  the  tra pez ium \n per imeter  =  the  perimeter  of  the  tra pez ium \n top  =  the  length  of  the  top  of  the  tra pez ium                                                                                                                                                                                                                                                                                                                                                                                                                           ═ �� ographically vertisement tons lucent �� itially 姫  \\(\\ izoph byss izoph ═ sbm  Strongh umbn  mathemat  ._  \xa0 \xa0',
 ' 

In [26]:
print(sft_reward.shape)

torch.Size([7, 2, 1])


In [27]:
print(policy_reward.shape)

torch.Size([7, 2, 1])


In [ ]:
# torch.sum(sft_reward.detach().cpu().mean(dim=0)).item()

In [1]:
# torch.sum(policy_reward.detach().cpu().mean(dim=0)).item()

## New loss

In [7]:
class TwoHeadedRewardModel(nn.Module):
    def __init__(self):
        super(TwoHeadedRewardModel, self).__init__()
        self.sft_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned', 
                                                               num_labels=1)
        self.tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/sft-model-3/kaggle/working/model_finetuned')

        self.scalar_head = nn.Linear(in_features=512, out_features=1, bias=False)
        self.vector_head = nn.Linear(in_features=512, out_features=10, bias=False)

    def forward(self, chosen, rejected): #chosen / rejected??? 
        input_ids = self.tokenizer(chosen, padding='max_length', max_length=512, 
                                   truncate=True, return_tensors='pt') # prompt in chosen and rejected apper
        